In [1]:
number_of_frames_to_record=500 #100000
background_chunk=100 #100
frame_width=848 #640
frame_height=480 #480
fps=30 #30
preset_json_file="d415paramset_848480.json"

In [2]:
import sys
sys.path.append('/usr/local/lib')
import gzip
import shutil
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib.colors as mcol
import matplotlib.cm as cm
import math
import imageio
import pickle
import gc
import os
abra=os.getcwd()
import time
import json
import pyrealsense2 as rs
import gzip
import cv2
from scipy import stats
from scipy import ndimage #for finding com
from scipy.misc import imsave
from itertools import cycle
cycol = cycle('bgrcmk') #here I'm generating a list of colors I can plot

In [3]:
  with open(preset_json_file) as json_data: #Json file to be loaded #NOTE im using default NOT the json that I just created above
    starting_params = json.load(json_data)
    starting_params=str(starting_params).replace("'", '\"') #IF YOU DON't DO THIS IT WILL NOT WORK with the librealsense sdk
    print(starting_params)

{"aux-param-autoexposure-setpoint": "400", "aux-param-colorcorrection1": "0.461914", "aux-param-colorcorrection10": "-0.553711", "aux-param-colorcorrection11": "-0.553711", "aux-param-colorcorrection12": "0.0458984", "aux-param-colorcorrection2": "0.540039", "aux-param-colorcorrection3": "0.540039", "aux-param-colorcorrection4": "0.208008", "aux-param-colorcorrection5": "-0.332031", "aux-param-colorcorrection6": "-0.212891", "aux-param-colorcorrection7": "-0.212891", "aux-param-colorcorrection8": "0.68457", "aux-param-colorcorrection9": "0.930664", "aux-param-depthclampmax": "65536", "aux-param-depthclampmin": "0", "aux-param-disparityshift": "0", "controls-autoexposure-auto": "True", "controls-autoexposure-manual": "33000", "controls-color-autoexposure-auto": "True", "controls-color-autoexposure-manual": "166", "controls-color-backlight-compensation": "0", "controls-color-brightness": "0", "controls-color-contrast": "50", "controls-color-gain": "64", "controls-color-gamma": "300", "co

In [4]:
#https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/align-depth2color.py

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)
stream_profile=profile.get_stream(rs.stream.depth)
# getting instrinsic parameters https://github.com/IntelRealSense/librealsense/issues/869
intrinsics = stream_profile.as_video_stream_profile().get_intrinsics() # Downcast to video_stream_profile and fetch intrinsics
print(intrinsics.ppx)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
#clipping_distance_in_meters = 2 #1 meter
#clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color #was previously aligned to color
align = rs.align(align_to)
count=0

415.46185302734375
Depth Scale is:  0.0010000000474974513


In [5]:
# Streaming loop

with open('pythondepthdata.bin','ab') as depthbinarydata, open('pythoncolordata.bin', 'ab') as colorbinarydata: #While the file called "pythonedpthdata.bin" is open for appending binary, 
    try:
        while count<number_of_frames_to_record:
            # Get frameset of color and depth
            frames = pipeline.wait_for_frames()
            # frames.get_depth_frame() is a 640x360 depth image

            # Align the depth frame to color frame
            aligned_frames = align.process(frames)

            # Get aligned frames
            aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            # Validate that both frames are valid
            if not aligned_depth_frame or not color_frame:
                continue

            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            depth_image.tofile(depthbinarydata) #save data as binarydata (specified by the with open as line)

            color_image = np.asanyarray(color_frame.get_data())
            color_image.tofile(colorbinarydata) #save data as binarydata (specified by the with open as line)
            
            if count==background_chunk:
                output=input("please insert calibration object and enter YYMMDD_exp######  ")
            if count==background_chunk+50:
                input("please insert object of interest")
            
            with open('depth_ts.txt', 'a') as ts:
                ts.write('%s\n' %rs.frame.get_timestamp(frames)) #write time stamp float as a string to a new line
            

            # Remove background - Set pixels further than clipping_distance to grey
            #grey_color = 153
            depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
            #bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

            # Render images
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.13), cv2.COLORMAP_JET)
            images = np.hstack((color_image, depth_colormap))
            count=count+1;
    finally:
        pipeline.stop()

please insert calibration object and enter YYMMDD_exp######  181031_mouse_two_cherkboards_1
please insert object of interestmouse


In [6]:
intr=print(intrinsics)

width: 848, height: 480, ppx: 415.462, ppy: 232.335, fx: 633.362, fy: 633.362, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]


In [7]:
os.mkdir(output)
shutil.move('pythondepthdata.bin',output)
shutil.move('pythoncolordata.bin',output)
shutil.move('depth_ts.txt',output)
with open(abra + '/' + output + '/' + output + '_variables', 'wb') as variables:
    pickle.dump([number_of_frames_to_record,frame_height,frame_width,fps,background_chunk,intr],variables)

In [8]:
os.chdir(output)
os.mkdir(output+'_depth_frames')
os.mkdir(output+'_color_frames')
os.mkdir(output+'_stacked_frames')

In [9]:

with open(output+'_variables', 'rb') as variables:
    number_of_frames,frame_height,frame_width,fps,background_chunk,intrinsics = pickle.load(variables) #added intrisincs
    
exp_numb=0
box_size=192
stringency=10 #10 for d415 50 for sr300
grand_frame=int(exp_numb*(number_of_frames-background_chunk))
min_range=0
max_range=300 #1000

In [10]:
frame_height=720
frame_width=1280
depthframes = np.fromfile('pythondepthdata.bin', dtype=np.uint16).reshape(-1, frame_height, frame_width)
depth_background_frames = depthframes[0:background_chunk]
depthframes=depthframes[background_chunk:number_of_frames]
depthbackgroundmean=np.median(depth_background_frames, axis=0) #may be better to use median
depthbackgroundvariance=np.var(depth_background_frames, axis=0)

In [11]:

colorframes = np.fromfile('pythoncolordata.bin', dtype=np.uint8).reshape(-1, frame_height, frame_width,3)


In [12]:
'''
depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
images = np.hstack((color_image, depth_colormap))
'''

'\ndepth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)\nimages = np.hstack((color_image, depth_colormap))\n'

In [42]:
###get the depth and color image of the calibration object
current_frame=40
depth_frame_of_interest=depthframes[current_frame] 
color_frame_of_interest=colorframes[current_frame+background_chunk] 
#color
#color_frame_of_interest=np.dot(color_frame_of_interest[...,:3], [.299, .587, .114])
#color_frame_of_interest=np.subtract(color_frame_of_interest,(colorbackgroundmean))
#DEPTH BACKGROUND SUBTRACTION#####################################################
#Background subtraction with no filters
    
#IF PIXEL ISN'T SIGNIFICANTLY DIFFERENT FROM THE BACKGROUND THEN DROP IT
backgroundmultiplier=np.abs(depth_frame_of_interest - depthbackgroundmean) - (stringency*np.sqrt(depthbackgroundvariance)) 
    

#WE EXPECT POSITIVE VALUES FOR HIGH DIFFERENCES make them stay by multiplying by 1
backgroundmultiplier[backgroundmultiplier>0]=1
#WE EXPECT NEGATIVE VALUES FOR SMALL OR NO DIFFERENCE probably just noise, get rid of them by multiplying by 0
backgroundmultiplier[backgroundmultiplier<=0]=0
       
# SUBTRACT BACKGROUND BY MULTIPLYING INSIGNIFIACNT FRAMES BY ZERO
depth_frame_of_interest_backgroundsubtracted=(np.multiply(depth_frame_of_interest,backgroundmultiplier)).astype('uint8') #IMWRITE TAKES UINT8
depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted>max_range]=0
depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted<=min_range]=0
'''

'''
depth_image=depth_frame_of_interest
color_image=color_frame_of_interest

In [43]:
num_calib_points=50
plt.figure(figsize=(40,20))
plt.imshow(color_image)
realpoints=np.rint(plt.ginput(num_calib_points,timeout=0))
virtualpoints=np.rint(plt.ginput(num_calib_points,timeout=0))
np.rint(realpoints)

array([[594., 516.],
       [593., 483.],
       [589., 451.],
       [586., 418.],
       [584., 385.],
       [579., 355.],
       [576., 325.],
       [571., 292.],
       [624., 505.],
       [618., 477.],
       [615., 446.],
       [613., 415.],
       [610., 386.],
       [606., 354.],
       [601., 327.],
       [600., 295.],
       [907., 303.],
       [912., 331.],
       [914., 360.],
       [919., 388.],
       [926., 420.],
       [930., 450.],
       [878., 307.],
       [881., 335.],
       [886., 365.],
       [892., 394.],
       [894., 424.],
       [900., 455.],
       [849., 309.],
       [853., 340.],
       [857., 368.],
       [861., 398.],
       [865., 428.],
       [870., 456.],
       [821., 315.],
       [821., 341.],
       [826., 371.],
       [833., 398.],
       [837., 431.],
       [839., 459.],
       [792., 318.],
       [793., 344.],
       [798., 376.],
       [803., 405.],
       [806., 433.],
       [812., 463.],
       [763., 323.],
       [767.,

In [44]:
twod_realpoints=realpoints.astype(int)
twod_virtualpoints=virtualpoints.astype(int)

In [45]:
threed_realpoints=np.zeros((num_calib_points,3),dtype=int)
threed_virtualpoints=np.zeros((num_calib_points,3),dtype=int)

for i in range(num_calib_points):
    rx=twod_realpoints[i][0]
    ry=twod_realpoints[i][1]
    threed_realpoints[i]=[twod_realpoints[i][1],twod_realpoints[i][0],depth_image[ry,rx]]
    
for i in range(num_calib_points):
    vx=twod_virtualpoints[i][0]
    vy=twod_virtualpoints[i][1]
    threed_virtualpoints[i]=[twod_virtualpoints[i][1],twod_virtualpoints[i][0],depth_image[vy,vx]]

In [46]:
threed_realpoints

array([[516, 594, 617],
       [483, 593, 620],
       [451, 589, 620],
       [418, 586, 622],
       [385, 584, 626],
       [355, 579, 626],
       [325, 576, 626],
       [292, 571, 621],
       [505, 624, 660],
       [477, 618, 658],
       [446, 615, 660],
       [415, 613, 660],
       [386, 610, 661],
       [354, 606, 669],
       [327, 601, 665],
       [295, 600, 668],
       [303, 907, 686],
       [331, 912, 684],
       [360, 914, 682],
       [388, 919, 683],
       [420, 926, 681],
       [450, 930, 681],
       [307, 878, 690],
       [335, 881, 686],
       [365, 886, 683],
       [394, 892, 683],
       [424, 894, 682],
       [455, 900, 681],
       [309, 849, 689],
       [340, 853, 686],
       [368, 857, 686],
       [398, 861, 683],
       [428, 865, 683],
       [456, 870, 681],
       [315, 821, 686],
       [341, 821, 685],
       [371, 826, 684],
       [398, 833, 683],
       [431, 837, 682],
       [459, 839, 681],
       [318, 792, 686],
       [344, 793

In [47]:
threed_virtualpoints

array([[ 408, 1185,  877],
       [ 388, 1181,  877],
       [ 365, 1176,  877],
       [ 342, 1174,  877],
       [ 320, 1169,  882],
       [ 297, 1164,  884],
       [ 275, 1160,  884],
       [ 252, 1159,  874],
       [ 414, 1136,  886],
       [ 390, 1132,  883],
       [ 370, 1129,  886],
       [ 348, 1128,  884],
       [ 325, 1123,  885],
       [ 305, 1119,  884],
       [ 282, 1114,  886],
       [ 260, 1113,  881],
       [ 292, 1023,  726],
       [ 320, 1030,  731],
       [ 347, 1032,  727],
       [ 374, 1037,  724],
       [ 403, 1040,  723],
       [ 430, 1047,  727],
       [ 292, 1032,  745],
       [ 319, 1036,  743],
       [ 344, 1042,  745],
       [ 372, 1046,  745],
       [ 398, 1050,  744],
       [ 426, 1053,  739],
       [ 293, 1040,  761],
       [ 317, 1045,  758],
       [ 345, 1050,  761],
       [ 373, 1055,  763],
       [ 398, 1059,  762],
       [ 423, 1062,  759],
       [ 295, 1048,  779],
       [ 318, 1052,  776],
       [ 345, 1058,  776],
 

In [48]:
#code for affine transformation using 4 points from each space
def solve_affine( space1, space2 ):
    x = np.transpose(np.matrix(space1))
    y = np.transpose(np.matrix(space2))
    # add ones on the bottom of x and y
    x = np.vstack((x,[np.ones(len(space1),dtype=int)]))
    y = np.vstack((y,[np.ones(len(space2),dtype=int)]))
    # solve for A2
    A2 = y * x.I
    # return function that takes input x and transforms it
    # don't need to return the 4th row
    return lambda x: (A2*np.vstack((np.matrix(x).reshape(3,1),1)))[0:3,:]

In [49]:
virtual2realaffine=solve_affine(threed_virtualpoints,threed_realpoints)

In [50]:
#IM ABLE TO GET PRETTY DAMN CLOSE TO THE ONE POINT THAT DID NOT GO INTO THE AFFINE TRANSFORM!!!
virtual2realaffine(threed_virtualpoints[num_calib_points-1])

matrix([[408.14755394],
        [769.12008505],
        [680.46947223]])

In [51]:
#%matplotlib inline

In [52]:
plt.figure(figsize=(20,8))
plt.imshow(images)

In [53]:
plt.imshow(depth_image)

In [54]:
#sanity check
x=1012
y=210
virtual2realaffine([y,x,depth_image[y,x]])
        

matrix([[  27.45712132],
        [1962.20460109],
        [ 434.95666305]])

In [55]:
plt.imshow(color_image)

In [117]:
###new depth and color images are of the object of interest.
stringency=15
current_frame=299
depth_frame_of_interest=depthframes[current_frame] 
color_frame_of_interest=colorframes[current_frame+background_chunk] 
#color
#color_frame_of_interest=np.dot(color_frame_of_interest[...,:3], [.299, .587, .114])
#color_frame_of_interest=np.subtract(color_frame_of_interest,(colorbackgroundmean))
#DEPTH BACKGROUND SUBTRACTION#####################################################
#Background subtraction with no filters
    
#IF PIXEL ISN'T SIGNIFICANTLY DIFFERENT FROM THE BACKGROUND THEN DROP IT
backgroundmultiplier=np.abs(depth_frame_of_interest - depthbackgroundmean) - (stringency*np.sqrt(depthbackgroundvariance)) 
    

#WE EXPECT POSITIVE VALUES FOR HIGH DIFFERENCES make them stay by multiplying by 1
backgroundmultiplier[backgroundmultiplier>0]=1
#WE EXPECT NEGATIVE VALUES FOR SMALL OR NO DIFFERENCE probably just noise, get rid of them by multiplying by 0
backgroundmultiplier[backgroundmultiplier<=0]=0
       
# SUBTRACT BACKGROUND BY MULTIPLYING INSIGNIFIACNT FRAMES BY ZERO
depth_frame_of_interest_backgroundsubtracted=(np.multiply(depth_frame_of_interest,backgroundmultiplier)).astype('uint8') #IMWRITE TAKES UINT8
depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted>max_range]=0
depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted<=min_range]=0
'''

'''
depth_image=depth_frame_of_interest
color_image=color_frame_of_interest


In [118]:
import pandas as pd
from pyntcloud import PyntCloud

In [119]:
#NO AFFINE TRANSFORM

positions=np.zeros(((1280*720),3), dtype=int)
colors=np.zeros(((1280*720),3), dtype=int)

index=0

for i in range(1280):
    for j in range(720):
        
        positions[index,0]=i
        positions[index,1]=j
        positions[index,2]=-1*depth_image[j,i] #height,width
        colors[index,0]=color_image[j,i][0]
        colors[index,1]=color_image[j,i][1]
        colors[index,2]=color_image[j,i][2]
        
        if depth_frame_of_interest_backgroundsubtracted[j,i]==0: #==0
            positions[index,0]=0
            positions[index,1]=0
            positions[index,2]=0
            colors[index,0]=0
            colors[index,1]=0
            colors[index,2]=0
        
        index+=1

In [120]:
#Apply affine transform to pixels higher than a certain threshold (where the mirror is)

affpositions=np.zeros(((1280*720),3), dtype=int)
affcolors=np.zeros(((1280*720),3), dtype=int)

index=0

for i in range(1280):
    for j in range(720):
        
        if i>1000 and j>100 and j<500: #i looked at the images by eye and found the mirror to be above this point. If I set J to a large number I create "reverse" world where everything is affine transformed
            affpositions[index,0]=virtual2realaffine([j,i,depth_image[j,i]])[1]
            affpositions[index,1]=virtual2realaffine([j,i,depth_image[j,i]])[0]
            affpositions[index,2]=virtual2realaffine([j,i,depth_image[j,i]])[2]*-1 #if using raw image then *-1 but if using background subtracted image then leave as b
        else:
            affpositions[index,0]=i
            affpositions[index,1]=j
            affpositions[index,2]=-1*depth_image[j,i] #height,width
            
        affcolors[index,0]=color_image[j,i][0]
        affcolors[index,1]=color_image[j,i][1]
        affcolors[index,2]=color_image[j,i][2]
        
        if depth_frame_of_interest_backgroundsubtracted[j,i]==0: #==0
            affpositions[index,0]=0
            affpositions[index,1]=0
            affpositions[index,2]=0
            affcolors[index,0]=0
            affcolors[index,1]=0
            affcolors[index,2]=0
            
        
        index+=1

In [121]:
points = pd.DataFrame(
    positions.astype(np.float32), 
    columns=['x', 'y', 'z'])

points['red']=colors[:,0].astype(np.uint8)
points['blue']=colors[:,2].astype(np.uint8)
points['green']=colors[:,1].astype(np.uint8)

cloud = PyntCloud(points)


In [122]:
affpoints = pd.DataFrame(
    affpositions.astype(np.float32), 
    columns=['x', 'y', 'z'])

affpoints['red']=affcolors[:,0].astype(np.uint8)
affpoints['blue']=affcolors[:,2].astype(np.uint8)
affpoints['green']=affcolors[:,1].astype(np.uint8)

affcloud = PyntCloud(affpoints)

In [123]:
cloud.plot(initial_point_size=6)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(4.882261276245117, 720.3687620162964, -5.982…

In [124]:
affcloud.plot(initial_point_size=6)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(4.241404056549072, 720.4446876049042, -5.708…